In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

from pyspark.sql import functions as F
from pyspark.sql import Column

In [ ]:
spark = SparkSession.builder.master("local[2]").appName("AoC_2022_4[2]").getOrCreate()

In [ ]:
inputs = spark.read.option("delimiter", ",").schema("first_pair string, second_pair string").csv("./data/aoc_4.txt")

ranges = [(F.split(f"{pos}_pair", "-")[idx]).cast("int").alias(f"{pos}_{idx + 1}") for pos in ["first", "second"] for idx in [0, 1]]
inputs = inputs.select(ranges)

In [ ]:
fully_contains = lambda x, y: ((F.col(f"{y}_1").between(F.col(f"{x}_1"), F.col(f"{x}_2"))) & 
                         (F.col(f"{y}_2").between(F.col(f"{x}_1"), F.col(f"{x}_2"))))
perfectly_overlapping = fully_contains("first", "second") | fully_contains("second", "first")

In [ ]:
inputs.select(F.sum(F.when(perfectly_overlapping, F.lit(1)).otherwise(0)).alias("result")).head()

In [ ]:
any_contains = lambda x, y: ((F.col(f"{y}_1").between(F.col(f"{x}_1"), F.col(f"{x}_2"))) |
                         (F.col(f"{y}_2").between(F.col(f"{x}_1"), F.col(f"{x}_2"))))
any_overlapping = any_contains("first", "second") | any_contains("second", "first")

In [ ]:
inputs.select(F.sum(F.when(any_overlapping, F.lit(1)).otherwise(0)).alias("result")).head()